In [1]:
#read file
import os, types
import pandas as pd
import numpy as np
import sklearn.metrics
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_9e53905096384939a3b02eee24cc81a3 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_9e53905096384939a3b02eee24cc81a3 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_9e53905096384939a3b02eee24cc81a3 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Your_api_key',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_9e53905096384939a3b02eee24cc81a3)

body = client_9e53905096384939a3b02eee24cc81a3.get_object(Bucket='jupiterenergypaymentplancampaign-donotdelete-pr-4mtst9zqeqmfqs',Key='farm_recommendation.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

farm_recommendation_data = pd.read_csv(body)
farm_recommendation_data.head(1)


,username,community_id,location,carbon_credit,carbon_medals,comment_products,farm_location,suited_product,rent_period,total_area,idle_area,recommend
0,cfcd208495d565ef66e7dff9f98764da,c1/c2/c3,Alaska/AK,1806,NaN,/p1/p2/p3/p4/p5/p6/p7,Delaware/DE,/p1/p2/p3/p4/p5/p6/p7,93,1905,1688,0


In [2]:
import pandas.api.types as tp
import numpy as np

'''
only select province/state from location data
'''
def preprocessing_location(csv_data):
    for i, row in csv_data.iterrows():
        user_location = row['location']
        state = user_location.split('/')[-1]
        farm_location = row['farm_location']
        farm = farm_location.split('/')[-1]
        csv_data.at[i, 'location'] = state
        csv_data.at[i, 'farm_location'] = farm
    return csv_data

'''
count all category values for a column 
'''
def aggregate_data(csv_data, header):
    res = set()
    for i, row in csv_data.iterrows():
        data = row[header]
        if type(data) == float and pd.isna(data):
            continue
        category = data.split('/')
        for c in category:
            res.add(c)
        row[header] = category
    res = sorted(list(set(res)))
    return res, csv_data

'''
count location value frequency
'''
def aggregate_location(csv_data, header):
    res = {}
    count = 0
    for i, row in csv_data.iterrows():
        data = row[header]
        if data not in res.keys():
            res[data] = count
            count += 1
    return res

In [3]:
csv_data = preprocessing_location(farm_recommendation_data)
location_dictionary = aggregate_location(csv_data, 'location')
community, csv_data = aggregate_data(csv_data, 'community_id')
comment_product, csv_data = aggregate_data(csv_data, 'comment_products')
medals, csv_data = aggregate_data(csv_data, 'carbon_medals')

In [4]:
'''
one_hot_encoding algorithm
'''
def one_hot_encoding(list_data, aggregate_data):
    encoding = ""
    if type(list_data) == float and pd.isna(list_data):
        return encoding
    for element in aggregate_data:
        if element in list_data:
            encoding = encoding + '1'
        else:
            encoding = encoding + '0'
    return int(encoding)

'''
apply one-hot-encoding to given column
'''
def apply_encoding(csv_data, header, aggregate):
    for i, row in csv_data.iterrows():
        raw = row[header]
        encoding = one_hot_encoding(raw, aggregate)
        csv_data.at[i, header] = encoding
    return csv_data

'''
apply one-hot-encoding to location column
'''
def apply_location_encoding(csv_data):
    for i, row in csv_data.iterrows():
        location = row['location']
        farm_location = row['farm_location']
        csv_data.at[i, 'location'] = location_dictionary[location]
        csv_data.at[i, 'farm_location'] = location_dictionary[farm_location]
    return csv_data

'''
standardize numerical variable
'''
def standardlize(csv_data, header):
    mean = csv_data[header].mean()
    sd = csv_data[header].std()
    for i, row in csv_data.iterrows():
        rent = row[header]
        standard = (rent - mean) / sd
        csv_data.at[i, header] = standard
    return csv_data

'''
fill missing values with mode
'''
def fill_with_mode(csv_data, community_mode, medals_mode, comment_mode, location_mode):
    for i, row in csv_data.iterrows():
        if row['community_id'] == '':
            csv_data.at[i, 'community_id'] = community_mode
        if row['carbon_medals'] == '':
            csv_data.at[i, 'carbon_medals'] = medals_mode
        if row['comment_products'] == '':
            csv_data.at[i, 'comment_products'] = comment_mode
        if row['suited_product'] == '':
            csv_data.at[i, 'suited_product'] = comment_mode
        if row['location'] == '':
            csv_data.at[i, 'location'] = location_mode
        if row['farm_location'] == '':
            csv_data.at[i, 'farm_location'] = location_mode
    return csv_data

'''
loop through a given column
'''
def loop_data(csv_data, attribute):
    for i, row in csv_data.iterrows():
        print(row[attribute])

In [5]:
csv_data = apply_encoding(csv_data, 'community_id', community) 
csv_data = apply_encoding(csv_data, 'carbon_medals', medals)
csv_data = apply_encoding(csv_data, 'comment_products', comment_product)
csv_data = apply_encoding(csv_data, 'suited_product', comment_product)
csv_data = apply_location_encoding(csv_data)

community_mode = 0
medals_mode = csv_data['carbon_medals'].mode()
comment_mode = 0
location_mode = csv_data['location'].mode()
csv_data = fill_with_mode(csv_data, community_mode, medals_mode, comment_mode, location_mode)
csv_data['community_id'] = csv_data['community_id'].astype(int)
# print(csv_data['community_id'].dtypes)
csv_data['location'] = csv_data['location'].astype(int)
csv_data['carbon_medals'] = csv_data['carbon_medals'].astype(int)
csv_data['comment_products'] = csv_data['comment_products'].astype(int)
csv_data['farm_location'] = csv_data['farm_location'].astype(int)
csv_data['suited_product'] = csv_data['suited_product'].astype(int)

In [184]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
def train_model(csv_data, test_size, random_state):
    Y = csv_data['recommend']
    csv_data = csv_data.drop(['recommend', 'username'], axis = 1)
    X_train, X_test, y_train, y_test = train_test_split(csv_data, Y, test_size = test_size, random_state = random_state)
    model = XGBClassifier(objective = 'binary:logitraw')
    model.fit(X_train, y_train)
    return model, X_train
model, X_train = train_model(csv_data, 0.33, 7)

In [6]:
'''
aggregate on farm and user rent relationship
'''
def get_user_farm_dict(gf_rent):
    user_farm_dic = {}
    farm_dic = {}
    farm_sequence = 0
    for i, row in gf_rent.iterrows():
        farm = row['farm_id']
        user = row['username']
        if user not in user_farm_dic.keys():
            user_farm_dic[user] = []
        user_farm_dic[user].append(farm)
        if farm not in farm_dic.keys():
            farm_dic[farm] = farm_sequence
            farm_sequence = farm_sequence + 1
    return user_farm_dic, farm_dic

In [7]:
'''
calculate cosine similarity
'''
def calculate_similarity(user_farm_matrix):
    return sklearn.metrics.pairwise.cosine_similarity(user_farm_matrix)

'''
calculate user similarity
'''
def get_similar_user(given_user, user_farm_matrix, user_dic):
    user = user_dic[given_user]
    user_matrix = user_farm_matrix[user]
    sim_dic = {}
    similarity_score = calculate_similarity(user_farm_matrix)[user]
    for i in range(0, len(similarity_score)):
        if i != user:
            score = similarity_score[i]
            if score not in sim_dic:
                sim_dic[score] = []
            sim_dic[score].append(i)
    sorted_similarity_score = sorted(sim_dic.keys(), reverse = True)
    similar_user = []
    for i in range(0, min(3, len(sorted_similarity_score))):
        score = sorted_similarity_score[i]
        for j in range(0, len(sim_dic[score])):
            similar_user.append(sim_dic[score][j])
    return similar_user


In [8]:

def get_user_matrix(user_farm_dic, gf_rent_data, farms):
    num_user = len(user_farm_dic.keys())
    num_farm = len(farms.keys())
    user_sequence = 0
    user_dic = {}
    user_farm_matrix = np.ndarray(shape = (num_user, num_farm), dtype = int)
    user_farm_matrix.fill(0)
    for user in user_farm_dic.keys():
        for f in user_farm_dic[user]:
            farm_number = farms[f]
            user_farm_matrix[user_sequence][farm_number] = 1
        user_dic[user] = user_sequence
        user_sequence = user_sequence + 1
    return user_farm_matrix, user_dic

In [9]:

def get_recall_set(user_farm_matrix, similar_user, given_user, user_dic):
    farm_not_rent = []
    user_number = user_dic[given_user]
    for user in similar_user:
        user_matrix = user_farm_matrix[user]
        for i in range(0, len(user_matrix)):
            rent = user_matrix[i]
            user_rent = user_farm_matrix[user_number][i]
            if user_rent == 0 and rent == 1:
                farm_not_rent.append(i)
    return farm_not_rent 
def check_user_rent(gf_rent, given_user):
    user_rent = gf_rent[gf_rent['username'] == given_user]
    #need to check if the cell is NA/null/empty string
    if not user_rent['farm_id'].empty:
        return True
    return False

In [12]:
body = client_9e53905096384939a3b02eee24cc81a3.get_object(Bucket='jupiterenergypaymentplancampaign-donotdelete-pr-4mtst9zqeqmfqs',Key='farm_rent.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

farm_rent = pd.read_csv(body)

In [13]:
#preprocessing the farm rent
user_farm_dic, farms = get_user_farm_dict(farm_rent)

In [14]:
user_farm_matrix, user_dic = get_user_matrix(user_farm_dic, farm_rent, farms)

In [15]:
similar_user = get_similar_user(1234, user_farm_matrix, user_dic)

[1]


In [ ]:
json_obj = {}
json_obj["username"] = "Jack"
json_obj['community_id'] = 1
json_obj['comment_products'] = ["Corn, Tomato"]
json_obj['carbon_medals'] = "1"
json_obj['carbon_credit'] = 100
json_obj['location'] = "Queens, New York, NY, USA"
json_obj['rent_list'] = []
json_obj['rent_list'].append({
    'rent_id': '1',
    'land_id': '1',
    'farm_id': '1',
    'username': 'Jack',
    'rent_price': '0.0000',
    'area': '100',
    'rent_start_time': '2021-07-00:00:00',
    'rent_end_time': '2021-07-00:00:00',
    'confirm_crops': 'corn tomato',
    'create_time': '2021-07-15:09:37',
    'modify_time': '2021-07-14:32:46',
    'farm_location': 'Austin,Texas'
})
json_data = json.dumps(json_obj)
get_data = json.loads(json_data)

In [18]:
train_attribute = ['community_id', 'location', 'carbon_credit', 'carbon_medals', 'comment_products', 'farm_location', 'suited_product', 'rent_period', 'total_area', 'idle_area']

In [19]:
farm_rent = farm_rent.rename(columns={"community": "community_id", "user location": "location", "comment product": "comment_products", "farm location": "farm_location", "rent period": "rent_period"})
farm_rent = farm_rent.rename(columns={"suited product": "suited_product"})
print(farm_rent.head())

   username  farm_id                 location community_id comment_products  \
0      1234     2263  121 5th Ave/New York/NY          A/B    potato/tomato   
1      5678     3652  121 6th Ave/New York/NY          B/C             corn   

   farm_location        suited_product  rent_period  total_area  idle_area  \
0  Rollingbay/WA  tomato/peas/brocoli           365          30         20   
1    Portland/FL              cucumber          182          20         10   

   carbon_credit carbon_medals  recommend  
0            600      m1/m2/m5          1  
1            800            m2          0  


In [20]:
def farm_id_index(farm_rent):
    dic = {}
    for i, row in farm_rent.iterrows():
        dic[i] = row['farm_id']
    return dic
farm_id_index = farm_id_index(farm_rent)

In [21]:
#select recall set
farm_not_rent = get_recall_set(user_farm_matrix, similar_user, 1234, user_dic)
pd_index = []
for index in farm_not_rent:
    pd_index.append(farm_rent.iloc[index])
farm_predict_set = pd.DataFrame(pd_index, columns = farm_rent.columns)

In [22]:
predict_farm_attribute = ['farm_location', 'suited_product', 'rent_period', 'total_area', 'idle_area']
for column in farm_predict_set.columns:
    if column not in predict_farm_attribute:
        farm_predict_set = farm_predict_set.drop(column, axis = 1)

In [157]:
#check if the user did not rent any farm before
rent_before = check_user_rent(farm_rent, 1234)

In [23]:
#need to confirm with backend
test_user = {'community_id': 'c1/c2', 'location': 'New York/NY', 'carbon_credit': '589', 'carbon_medals': 'm1/m2', 'comment_products': 'p1'}
test_user = pd.DataFrame(test_user, index = [0])
#preprocessing the test user
def preprocess_data(csv_data, community, medals, comment_product):
    csv_data = apply_encoding(csv_data, 'community_id', community) 
    csv_data = apply_encoding(csv_data, 'carbon_medals', medals)
    csv_data = apply_encoding(csv_data, 'comment_products', comment_product)
    csv_data = apply_encoding(csv_data, 'suited_product', comment_product)
    csv_data = apply_location_encoding(csv_data)
    community_mode = 0
    medals_mode = csv_data['carbon_medals'].mode()
    comment_mode = 0
    location_mode = csv_data['location'].mode()
    csv_data = fill_with_mode(csv_data, community_mode, medals_mode, comment_mode, location_mode)
    csv_data['community_id'] = csv_data['community_id'].astype(int)
    # print(csv_data['community_id'].dtypes)
    csv_data['location'] = csv_data['location'].astype(int)
    csv_data['carbon_medals'] = csv_data['carbon_medals'].astype(int)
    csv_data['carbon_credit'] = csv_data['carbon_credit'].astype(int)
    csv_data['comment_products'] = csv_data['comment_products'].astype(int)
    csv_data['farm_location'] = csv_data['farm_location'].astype(int)
    csv_data['suited_product'] = csv_data['suited_product'].astype(int)
    return csv_data

In [24]:
def preprocessing_test_user_location(csv_data):
    for i, row in csv_data.iterrows():
        user_location = row['location']
        state = user_location.split('/')[-1]
        farm_location = row['farm_location']
        farm = user_location.split('/')[-1]
        csv_data.at[i, 'location'] = state
        csv_data.at[i, 'farm_location'] = farm
    return csv_data

In [25]:
def fill_with_test_mode(csv_data, community_mode, medals_mode, comment_mode, location_mode):
    for i, row in csv_data.iterrows():
        if row['community_id'] == '':
            csv_data.at[i, 'community_id'] = community_mode
        if row['carbon_medals'] == '':
            csv_data.at[i, 'carbon_medals'] = medals_mode
        if row['comment_products'] == '':
            csv_data.at[i, 'comment_products'] = comment_mode
        if row['location'] == '':
            csv_data.at[i, 'location'] = location_mode
    return csv_data

In [26]:
def apply_test_user_location_encoding(csv_data):
    for i, row in csv_data.iterrows():
        location = row['location']
        csv_data.at[i, 'location'] = location_dictionary[location]
    return csv_data

In [27]:
def change_index(csv_data1, farm_not_rent):
    csv_data1 = csv_data1.rename(mapper = {csv_data1.index.values[0]: farm_not_rent}, axis = 0)
    return csv_data1

In [28]:
csv_data1 = test_user
csv_data1 = change_index(csv_data1, farm_not_rent[0])
concate_rows = pd.concat([csv_data1, farm_predict_set], axis = 1)
if len(farm_not_rent) > 1:
    for rent in farm_not_rent[1:]:
        csv_data1 = change_index(csv_data1, rent)
        concate_rows.append(pd.concat([csv_data1, farm_predict_set], axis = 1))
        predict_set.append(concate_rows)
print(concate_rows)

  community_id     location carbon_credit carbon_medals comment_products  \
1        c1/c2  New York/NY           589         m1/m2               p1   

  farm_location suited_product  rent_period  total_area  idle_area  
1   Portland/FL       cucumber          182          20         10  


In [29]:
csv_data1 = preprocessing_location(concate_rows)
location_dictionary = aggregate_location(csv_data1, 'location')
community1, csv_data1 = aggregate_data(csv_data1, 'community_id')
comment_product1, csv_data1 = aggregate_data(csv_data1, 'comment_products')
farm_product1, csv_data1 = aggregate_data(csv_data1, 'suited_product')
medals1, csv_data1 = aggregate_data(csv_data1, 'carbon_medals')

In [30]:
print(csv_data1)

  community_id location carbon_credit carbon_medals comment_products  \
1        c1/c2       NY           589         m1/m2               p1   

  farm_location suited_product  rent_period  total_area  idle_area  
1            NY       cucumber          182          20         10  


In [31]:
csv_data1 = preprocess_data(csv_data1, community, medals, comment_product)

In [168]:
prediction = model.predict_proba(csv_data1)

[[0.9972397  0.00276029]]


In [170]:
def sort_farm(prediction, farm_not_rent):
    farm_score = {}
    for i in range(0, len(prediction)):
        pred_score = prediction[i]
        if pred_score[1] > pred_score[0]:
            farm_score[farm_not_rent[i]] = pred_score[1]
    sorted_farm_score = [key for key, value in sorted(farm_score.items(), key=lambda i: i[1], reverse=True)]
    return sorted_farm_score

In [180]:
sorted_farm_score = sort_farm(prediction, farm_not_rent)

[]


In [ ]:
def farm_recommendation(sorted_farm_score):
    count = 0
    recommended_farm = []
    for farm in sorted_farm_score:
        if count < 2:
            recommended_farm.append(farm_id_index[farm])
            count += 1
    return recommended_farm

In [ ]:
def feature():
    im=pd.DataFrame({'importance':model.feature_importances_})
    im=im.sort_values(by='importance',ascending=False)
    feature_index = im.index[0:3]
    # print(feature_index)
    features = [X_train.columns[x] for x in feature_index]
    return features